## 1. Import stock ticker list from FMP to SQLite3

Objectives:
- Get the full list of stock ticker available in FMP.
- Filter to get only stocks (not ETF's or Mutual funds) .
- Download list to SQLite3

In [1]:
# Imports
import os
import requests
import pandas as pd


In [2]:
# FMP API Key
api = os.environ.get("token_finmodelprep2")

# Import Ticker List
www = 'https://financialmodelingprep.com/api/v3'
stock_list = '/stock/list'
tickers = requests.get(f'{www}{stock_list}?apikey={api}').json()

# Create a DataFrame from import
stock_list_df = pd.DataFrame(tickers)

# Display pandas dataframes in full form
pd.set_option("display.max_rows", None, "display.max_columns", None)


In [3]:
# Show the Table
stock_list_df.head()


,symbol,name,price,exchange,exchangeShortName,type
0,SPY,SPDR S&P 500 ETF Trust,466.57,New York Stock Exchange Arca,AMEX,etf
1,CMCSA,Comcast Corporation,47.71,Nasdaq Global Select,NASDAQ,stock
2,KMI,"Kinder Morgan, Inc.",15.93,New York Stock Exchange,NYSE,stock
3,INTC,Intel Corporation,50.00,Nasdaq Global Select,NASDAQ,stock
4,MU,"Micron Technology, Inc.",84.35,Nasdaq Global Select,NASDAQ,stock


In [4]:
# Filter DF to display only stocks
rslt_df = stock_list_df[stock_list_df['type'] == "stock"]
rslt_df = rslt_df[rslt_df['exchangeShortName'] != "ETF"]
rslt_df = rslt_df[rslt_df['exchangeShortName'] != "MUTUAL_FUND"]
rslt_df.head(10)


,symbol,name,price,exchange,exchangeShortName,type
1,CMCSA,Comcast Corporation,47.71,Nasdaq Global Select,NASDAQ,stock
2,KMI,"Kinder Morgan, Inc.",15.93,New York Stock Exchange,NYSE,stock
3,INTC,Intel Corporation,50.00,Nasdaq Global Select,NASDAQ,stock
4,MU,"Micron Technology, Inc.",84.35,Nasdaq Global Select,NASDAQ,stock
6,GE,General Electric Company,93.13,New York Stock Exchange,NYSE,stock
7,BAC,Bank of America Corporation,43.58,New York Stock Exchange,NYSE,stock
10,AAPL,Apple Inc.,175.74,Nasdaq Global Select,NASDAQ,stock
11,MSFT,Microsoft Corporation,339.40,Nasdaq Global Select,NASDAQ,stock
12,SIRI,Sirius XM Holdings Inc.,6.29,Nasdaq Global Select,NASDAQ,stock
13,HPQ,HP Inc.,35.28,New York Stock Exchange,NYSE,stock


In [6]:
# Exploring the data

print("Number of Rows in the Database:")
print(len(rslt_df.index))
print("\n")

print("Exchanges Covered (Short):")
print(rslt_df.exchangeShortName.unique())
print("\n")

print("Exchanges Covered (Long):")
print(rslt_df.exchange.unique())
print("\n")

print("Types:")
print(rslt_df.type.unique())
print("\n")


Number of Rows in the Database:
25792


Exchanges Covered (Short):
['NASDAQ' 'NYSE' 'AMEX' 'EURONEXT' 'TSX' 'MCX' 'XETRA' 'NSE' 'LSE' 'SIX'
 'HKSE' 'OSE' 'BRU' 'JKT' 'VIE' 'SGO' 'SHZ' 'SHH' 'HAM' 'CPH' 'ATH' 'MIL'
 'JPX' 'KSC' 'KOE' 'STO' 'IST' 'TAI' 'MEX' 'JNB' 'SAT' 'LIS' '' 'SET'
 'EBS' 'IOB' 'TLV' 'WSE' 'DOH' 'KLS' 'PRA' 'AMS' 'BER' 'TWO' 'SAU' 'HEL'
 'ICE']


Exchanges Covered (Long):
['Nasdaq Global Select' 'New York Stock Exchange' 'Nasdaq Capital Market'
 'New York Stock Exchange Arca' 'Nasdaq Global Market'
 'NASDAQ Global Market' 'Other OTC' 'Paris' 'Amsterdam' 'Brussels'
 'Lisbon' 'Toronto' 'YHD' 'EURONEXT' 'Swiss' 'AMEX' 'MCX' 'XETRA' 'NSE'
 'LSE' 'SIX' 'HKSE' 'OSE' 'NASDAQ' 'Sao Paolo' 'TSXV' 'Frankfurt' 'HKG'
 'NCM' 'MCE' 'ASE' 'OSL' 'Oslo' 'FGI' 'Irish' 'Canadian Sec' 'NZSE'
 'Nasdaq' 'Jakarta' 'Vienna' 'Santiago' 'Shenzhen' 'Shanghai' 'Hamburg'
 'Copenhagen' 'Helsinki' 'Athens' 'Milan' 'Tokyo' 'KSE' 'KOSDAQ'
 'Stockholm' 'Istanbul' 'Taiwan' 'Mexico' 'Johannesburg' 'SAT'

Save to SQlite

In [5]:
# Saving to SQLite3
from sqlalchemy import create_engine

# Create an engine
engine = create_engine('sqlite:///financialmodelingprep.db', echo=False)
sqlite_connection = engine.connect()
# sqlite is the database type
# financialmodelingprep.db is the sqlite db name
# echo=True to see output from database connection

# Create a table
sqlite_table = "stock_list" 
rslt_df.to_sql(sqlite_table, sqlite_connection, if_exists='replace', index=False)
# Alternativas:
    # if_exists: ‘fail’, ‘replace’, ‘append’
        # fail: Raise a ValueError
        # replace: Drop the table before inserting new values.
        # append: Insert new values to the existing table
    # index: bool, default True.
        # Write DataFrame index as a column. 
        # Uses index_label as the column name in the table.

# Close connection
sqlite_connection.close()
